In [2]:
# Do all relevant imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sps
import time
%matplotlib inline
print("Libs imported.")

Libs imported.


In [3]:
a = sps.coo_matrix(np.random.rand(5,5))
print(a)

  (0, 0)	0.505095922053
  (0, 1)	0.466373676232
  (0, 2)	0.994053483323
  (0, 3)	0.437291570171
  (0, 4)	0.823267553324
  (1, 0)	0.223245941425
  (1, 1)	0.874722963906
  (1, 2)	0.840487628015
  (1, 3)	0.19749195933
  (1, 4)	0.447722705284
  (2, 0)	0.80813155954
  (2, 1)	0.647149318753
  (2, 2)	0.690885608728
  (2, 3)	0.0962691907892
  (2, 4)	0.119718355233
  (3, 0)	0.360579434437
  (3, 1)	0.742784714683
  (3, 2)	0.887075414726
  (3, 3)	0.32505642688
  (3, 4)	0.620661729493
  (4, 0)	0.944086838331
  (4, 1)	0.144928626994
  (4, 2)	0.39102764688
  (4, 3)	0.949844134697
  (4, 4)	0.90032815039


In [3]:
a = a.tocsr()
print(isinstance(a, sps.csr_matrix))

b = a * a.T

print(isinstance(a, sps.csr_matrix))


True
True


In [4]:
a = sps.coo_matrix(np.random.rand(10000,10000))
f = np.random.choice(a = [True,False], size = 10000, p = [0.5, 0.5])

csr = a.tocsr()
cp = time.time()
print(csr[:,f].shape)
print("csr: %s "%(time.time()-cp))

csc = a.tocsc()
cp = time.time()
print(csc[:,f].shape)
print("csc: %s "%(time.time()-cp))


(10000, 4986)
csr: 1.8011391162872314 
(10000, 4986)
csc: 0.505141019821167 


In [5]:
csc.shape

(10000, 10000)

In [20]:
a = sps.csr_matrix(np.random.rand(5,5))
f = sps.csr_matrix([True, False, True, False, True])
print(a[f])

[[ 0.29385724  0.06505705  0.36877768]]


In [18]:
def top_k(M, k):
    starttime = time.time()

    #mask = np.zeros(M.shape, dtype = bool)
    rows = np.zeros((k*M.shape[0],), dtype = int)
    cols = np.zeros((k*M.shape[0],), dtype = int)
    vals = np.zeros((k*M.shape[0],), dtype = float)

    i = 0
    cp = time.time()
    M = M.todense()
    print("Converted matrix to dense, %s sec." %(time.time()-cp))
    
    for track_similarities in M:      
        # Get top K indices
        top_k_idx = np.argsort(track_similarities)[0,-k:]
        
        # Create filter
        rows[k*i:k*(i+1)] = [i]*k
        cols[k*i:k*(i+1)] = top_k_idx
        top_k_vals = track_similarities[0, top_k_idx]
        vals[k*i:k*(i+1)a] = top_k_vals
        
        
        i += 1

    M_top_k = sps.coo_matrix((vals, (rows, cols)), shape = M.shape, dtype = float).tocsr()
    print("Computed M_top_k matrix in %s sec. "%(time.time()-starttime))
    return M_top_k


a = sps.csr_matrix(np.random.rand(20000,20000))
#print(a)
cp = time.time()
print(top_k(a, 5).nnz)
print("%s sec" %(time.time()-cp))

Converted matrix. 9.053355932235718 sec. 9.05438780784607 from start
(20000, 20000)
100000
38.22781276702881 sec


In [5]:
%load_ext Cython


In [17]:
%%cython
import numpy as np
import time
cimport numpy as np
from cython.view cimport array as cvarray

cdef get_rcv(M, k_in):
    #Takes dense matrix, returns rows, cols, vals. 
    
    # Initiate lenghts. 
    cdef int nrows = M.shape[0]
    cdef int ncols = M.shape[1]
    
    cdef i, j, 
    cdef int k = k_in
    
    cdef np.ndarray[np.int32_t, ndim=1] rows = np.zeros(M.shape[0]*k, dtype=np.int32)
    cdef np.ndarray[np.int32_t, ndim=1] cols = np.zeros(M.shape[1]*k, dtype=np.int32)
    cdef np.ndarray[np.float32_t, ndim=1] vals = np.zeros(M.shape[0]*k, dtype=np.float32)

    cdef np.ndarray[np.float32_t, ndim=1] top_k_vals = np.zeros(k, dtype=np.float32)

    

    
    narr = np.zeros(M.shape[1], dtype=np.float64)
    cdef double [:] track_similarities = narr
    
    narr = np.zeros(k, dtype=np.int32)
    cdef int [:] top_k_idx = narr

    
    for i in range(nrows):
        track_similarities = M[i,:]
        print("asd")
        # Get top K indices
        top_k_idx = np.argsort(track_similarities)[-k:]
        print(top_k_idx)
        # Create new matrix        
        for j in range(k): 
            rows[k*i+j] = i

        
        for j in range(len(top_k_idx)): 
            cols[k*i+j] = top_k_idx[j]
        
            
        for j in range(k): 
            top_k_vals[j] = track_similarities[top_k_idx[j]]
        
        
        for j in range(len(top_k_vals)):
            vals[k*i+j] = top_k_vals[j]

        
        i = i+1
        
        if (i % 100 == 0):
            print(i)

    return rows, cols, vals
cp = time.time()
a = np.random.rand(1000,1000)
get_rcv(a, 20)
print(time.time()-cp)

AttributeError: type object '_cython_magic_3ebe84f59a8ee294e7c2d7eb3f565609.arr' has no attribute '__reduce_cython__'

In [93]:
1k: 0.07 s
10k: 8 s
20k: 36 s



SyntaxError: invalid syntax (<ipython-input-93-b6b684354e96>, line 1)